# Benchmarking docking and scoring methods with PLEX
## Overview
In this notebook we are running two docking models against the PDBBind benchmark. 
* equibind
* diffdock

We compare the performance of each method using the commonly used RSMD metric for 3D ligand position. 
Taking these models a step further, we combine their pose prediction capability with existing, ML-based, scoring functions, such as ODDT.

## Requirements
In order to run this notebook, you will need: 
* PLEX installed on your device
* PDBBind benchmark data downloaded from [Stärk et al.](https://zenodo.org/record/6408497)
* PDBBind affinity data downloaded from the official [website](https://pdbbind.oss-cn-hangzhou.aliyuncs.com/download/PDBbind_v2020_plain_text_index.tar.gz)

## Learn more 
Head to our [docs](docs.labdao.xyz) to learn more about how to install, use, and contribute to PLEX.

## PLEX setup

In [93]:
import os
import sys
import importlib

# this can disapear once plex is a pip package
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import plex.sdk
importlib.reload(plex.sdk)

os.environ["PLEX_ACCESS_TOKEN"] = "mellon"
os.environ["PLEX_ENV"] = "stage"

## Generating equibind poses
### Generating IO objects for the PDBBind benchmark

In [50]:
import csv
import os
import json

def create_pdbind_io_dict(csv_path):
    io_data = []
    
    with open(csv_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        
        for row in csvreader:
            protein_path = os.path.join("/home/ubuntu/", row['protein_path'])
            ligand_path = os.path.join("/home/ubuntu/", row['ligand_description'])
            
            if not os.path.exists(protein_path) or not os.path.exists(ligand_path):
                print(f"Skipping row {row['complex_name']} due to missing file(s).")
                continue
            
            entry = {
                "tool": "tools/equibind.json",
                "inputs": {
                    "protein": {
                        "class": "File",
                        "filepath": protein_path
                    },
                    "small_molecule": {
                        "class": "File",
                        "filepath": ligand_path
                    }
                },
                "outputs": {
                    "best_docked_small_molecule": {
                        "class": "File",
                        "filepath": ""
                    },
                    "protein": {
                        "class": "File",
                        "filepath": ""
                    }
                },
                "state": "created",
                "errMsg": ""
            }
            
            io_data.append(entry)
    
    return io_data

# Example usage
csv_path = '/home/ubuntu/datasets/diffdock_testdata.csv'
io_sig = create_pdbind_io_dict(csv_path)


In [ ]:
print(io_sig)

### Running inference

In [ ]:
from plex.sdk import run_plex

run_plex(io_sig, concurrency=2); # remove semicolon to display outputs

## Generate Diffdock poses
### Generating IO objects for the PDBind benchmark

In [95]:
import csv
import os
import json

def create_pdbind_io_dict(csv_path):
    io_data = []
    
    with open(csv_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        
        for row in csvreader:
            protein_path = os.path.join("/home/ubuntu/", row['protein_path'])
            ligand_path = os.path.join("/home/ubuntu/", row['ligand_description'])
            
            if not os.path.exists(protein_path) or not os.path.exists(ligand_path):
                print(f"Skipping row {row['complex_name']} due to missing file(s).")
                continue
            
            entry = {
                "tool": "tools/diffdock.json",
                "inputs": {
                  "protein": {
                    "type": "File",
                    "filepath": protein_path
                  },
                  "small_molecule": {
                    "type": "File",
                    "filepath": ligand_path
                  },
                },
                "outputs": {
                  "best_docked_small_molecule": {
                    "type": "File",
                    "item": "",
                    "glob": ["index*/rank1.sdf"]
                  },
                  "all_docked_small_molecules": {
                    "type": "Array",
                    "item": "File",
                    "glob": ["index*/rank*.sdf"]
                  },
                  "protein": {
                    "type": "File",
                    "item": "",
                    "glob": ["*.pdb"]
                  }
                }
            }
            
            io_data.append(entry)
    
    return io_data

# Example usage
csv_path = '/home/ubuntu/datasets/diffdock_testdata.csv'
io_diffdock_graph = create_pdbind_io_dict(csv_path)

In [ ]:
from plex.sdk import run_plex

run_plex(io_diffdock_graph, concurrency=2, local=True) # remove semicolon to display outputs

Plex version (v0.6.1) up to date.
BACALHAU_API_HOST not set, using default host
toolPath 
Running IPWL io path
Created job directory:  /home/ubuntu/plex/24ead1c9-d2d5-479d-b09a-cb1bdb84e73c
Reading IO Entries from:  /tmp/tmpzb349e58/io_data.json
Initialized IO file at:  /home/ubuntu/plex/24ead1c9-d2d5-479d-b09a-cb1bdb84e73c/io.json
Processing IO Entries
Starting to process IO entry 5 
Starting to process IO entry 18 
Success processing IO entry 5 
Starting to process IO entry 2 
Success processing IO entry 18 
Starting to process IO entry 0 
Success processing IO entry 2 
Starting to process IO entry 1 
Success processing IO entry 0 
Starting to process IO entry 11 
Success processing IO entry 1 
Starting to process IO entry 14 
Success processing IO entry 11 
Starting to process IO entry 13 
Success processing IO entry 14 
Starting to process IO entry 9 
Success processing IO entry 13 
Starting to process IO entry 49 
Success processing IO entry 9 
Starting to process IO entry 19 
Suc

### Run statistics

In [ ]:
# generating statistics on the success rate of the runs
import json
import pandas as pd

def get_state_counts(json_filepath):
    # Load the JSON data from the file
    with open(json_filepath, 'r') as f:
        data = json.load(f)
    
    # Extract the "state" and "errMsg" values from each JSON object
    state_errMsg_list = [{'state': item['state'], 'errMsg': item['errMsg']} for item in data]
    
    # Convert the list of dictionaries to a Pandas DataFrame
    df = pd.DataFrame(state_errMsg_list)
    
    # Count the occurrences of each unique "state" and "errMsg" combination
    counts_df = df.groupby(['state', 'errMsg']).size().reset_index(name='count')
    
    return counts_df, df

# Example usage
json_filepath = '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/io.json'
state_counts_df, complete_df = get_state_counts(json_filepath)
print(state_counts_df)

In [ ]:
complete_df[complete_df['state'] == 'failed']

### Resubmitting failed tasks

In [ ]:
def resubmit_failed_states(json_filepath):
    # Load the JSON data from the file
    with open(json_filepath, 'r') as f:
        data = json.load(f)
    
    # Filter the JSON list to include only entries with a failed state
    failed_entries = [entry for entry in data if entry['state'] == 'failed']
    
    # Create the io_sig object for each failed entry
    io_sig = []
    for entry in failed_entries:
        # Extract the relevant information from the JSON entry
        tool = entry['tool']
        inputs = entry['inputs']
        outputs = entry['outputs']
        state = 'created'  # Set the state to 'created' for resubmission
        errMsg = ''
        
        # Create a new entry for the io_sig object
        new_entry = {
            'tool': tool,
            'inputs': inputs,
            'outputs': outputs,
            'state': state,
            'errMsg': errMsg
        }
        
        # Append the new entry to the io_sig object
        io_sig.append(new_entry)
    
    return io_sig

# Example usage
json_filepath = '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/io.json'
io_sig = resubmit_failed_states(json_filepath)


In [ ]:
from plex.sdk import run_plex

run_plex(io_sig, concurrency=6)

In [ ]:
print(complete_df)

In [ ]:
### 

In [ ]:
run_plex(io_sig, concurrency=6)

## Benchmarking predicted Binding Pose

In [73]:
# we are loading the io dict of the docking run
json_filepath = '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/io.json'

with open(json_filepath, 'r') as file:
    json_dict = json.load(file)

# Print the first entry of the dictionary
print(json_dict[:3])

[{'outputs': {'best_docked_small_molecule': {'class': 'File', 'filepath': ''}, 'protein': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-0/outputs/6qqw_protein_processed.pdb'}}, 'tool': 'tools/equibind.json', 'inputs': {'protein': {'class': 'File', 'filepath': '/home/ubuntu/data/PDBBind_processed/6qqw/6qqw_protein_processed.pdb'}, 'small_molecule': {'class': 'File', 'filepath': '/home/ubuntu/data/PDBBind_processed/6qqw/6qqw_ligand.mol2'}}, 'state': 'failed', 'errMsg': 'no output data found for: [best_docked_small_molecule]'}, {'outputs': {'best_docked_small_molecule': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-1/outputs/6d08_protein_processed_6d08_ligand_docked.sdf'}, 'protein': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-1/outputs/6d08_protein_processed.pdb'}}, 'tool': 'tools/equibind.json', 'inputs': {'protein': {'class': 'File', 'filepath': '/

In [78]:
def create_oddt_io_dict(template, prev_dict):
    # Copy the template to avoid modifying the original
    new_dict = template.copy()

    # Replace the filepaths in the new dictionary with the values from the previous dictionary
    new_dict['inputs']['protein']['filepath'] = prev_dict['outputs']['protein']['filepath']
    new_dict['inputs']['small_molecule']['filepath'] = prev_dict['outputs']['best_docked_small_molecule']['filepath']

    return new_dict

# Example usage
template = {
    "outputs": {
        "scored_small_molecule": {
            "class": "File",
            "filepath": ""
        },
        "scores": {
            "class": "File",
            "filepath": ""
        }
    },
    "tool": "tools/oddt.json",
    "inputs": {
        "protein": {
            "class": "File",
            "filepath": "/Users/rindtorff/github/labdao/plex/testdata/scoring/abl/7n9g.pdb"
        },
        "small_molecule": {
            "class": "File",
            "filepath": "/Users/rindtorff/github/labdao/plex/testdata/scoring/abl/7n9g_ZINC000003986735_docked.sdf"
        }
    },
    "state": "processing",
    "errMsg": ""
}

prev_dict = {
    'outputs': {
        'best_docked_small_molecule': {
            'class': 'File',
            'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-1/outputs/6d08_protein_processed_6d08_ligand_docked.sdf'
        },
        'protein': {
            'class': 'File',
            'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-1/outputs/6d08_protein_processed.pdb'
        }
    },
    'tool': 'tools/equibind.json',
    'inputs': {
        'protein': {
            'class': 'File',
            'filepath': '/home/ubuntu/data/PDBBind_processed/6d08/6d08_protein_processed.pdb'
        },
        'small_molecule': {
            'class': 'File',
            'filepath': '/home/ubuntu/data/PDBBind_processed/6d08/6d08_ligand.sdf'
        }
    },
    'state': 'completed',
    'errMsg': ''
}

new_dict = create_oddt_io_dict(template, prev_dict)
print(new_dict)


{'outputs': {'scored_small_molecule': {'class': 'File', 'filepath': ''}, 'scores': {'class': 'File', 'filepath': ''}}, 'tool': 'tools/oddt.json', 'inputs': {'protein': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-1/outputs/6d08_protein_processed.pdb'}, 'small_molecule': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-1/outputs/6d08_protein_processed_6d08_ligand_docked.sdf'}}, 'state': 'processing', 'errMsg': ''}


In [81]:
def create_oddt_io_dict_list(template, prev_dict_list):
    new_dict_list = []

    for prev_dict in prev_dict_list:
        # Only process dictionaries where the state is 'completed'
        if prev_dict['state'] == 'completed':
            new_dict = create_oddt_io_dict(template, prev_dict)
            new_dict_list.append(new_dict)

    return new_dict_list

# Example usage
template = {
    "outputs": {
        "scored_small_molecule": {
            "class": "File",
            "filepath": ""
        },
        "scores": {
            "class": "File",
            "filepath": ""
        }
    },
    "tool": "tools/oddt.json",
    "inputs": {
        "protein": {
            "class": "File",
            "filepath": "/Users/rindtorff/github/labdao/plex/testdata/scoring/abl/7n9g.pdb"
        },
        "small_molecule": {
            "class": "File",
            "filepath": "/Users/rindtorff/github/labdao/plex/testdata/scoring/abl/7n9g_ZINC000003986735_docked.sdf"
        }
    },
    "state": "created",
    "errMsg": ""
}

# prev_dict_list should be the list of dictionaries you provided earlier
new_dict_list = create_oddt_io_dict_list(template, json_dict)

#for new_dict in new_dict_list:
#    print(new_dict)
print(new_dict_list[:5])

[{'outputs': {'scored_small_molecule': {'class': 'File', 'filepath': ''}, 'scores': {'class': 'File', 'filepath': ''}}, 'tool': 'tools/oddt.json', 'inputs': {'protein': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-360/outputs/6d3x_protein_processed.pdb'}, 'small_molecule': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-360/outputs/6d3x_protein_processed_6d3x_ligand_docked.sdf'}}, 'state': 'created', 'errMsg': ''}, {'outputs': {'scored_small_molecule': {'class': 'File', 'filepath': ''}, 'scores': {'class': 'File', 'filepath': ''}}, 'tool': 'tools/oddt.json', 'inputs': {'protein': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-360/outputs/6d3x_protein_processed.pdb'}, 'small_molecule': {'class': 'File', 'filepath': '/home/ubuntu/plex/0e1b24c5-870e-4a58-9b61-a302cecbbcd0/entry-360/outputs/6d3x_protein_processed_6d3x_ligand_docked.sdf'}}, 'state': 'create

In [87]:
from plex.sdk import run_local
run_local(new_dict_list)

Plex version (v0.6.1) up to date.
BACALHAU_API_HOST not set, using default host
toolPath 
Running IPWL io path
Created job directory:  /home/ubuntu/plex/9729e9a5-b3f7-4f5a-86d8-ebc723674370
Reading IO Entries from:  /tmp/tmpnmcf3y25/io_data.json
Initialized IO file at:  /home/ubuntu/plex/9729e9a5-b3f7-4f5a-86d8-ebc723674370/io.json
Processing IO Entries
Starting to process IO entry 7 
Generated docker cmd: docker run  -v /home/ubuntu/plex/9729e9a5-b3f7-4f5a-86d8-ebc723674370/entry-7/inputs:/inputs -v /home/ubuntu/plex/9729e9a5-b3f7-4f5a-86d8-ebc723674370/entry-7/outputs:/outputs quay.io/labdao/openbabel@sha256:1087315d7eda6d0632c9f9df72500ab9f6fef612c79bae7410473a2336f7be34 /bin/bash -c "echo 'reference,comparison,RMSD' > /outputs/rmsd.csv && echo -n '6d3x_ligand,6d3x_protein_processed_6d3x_ligand_docked,' > /outputs/temp.csv && obrms -firstonly /inputs/6d3x_ligand.sdf /inputs/6d3x_protein_processed_6d3x_ligand_docked.sdf | awk '{print $2}' | tr -d '\n' >> /outputs/temp.csv && cat /out

## Benchmarking predicted Binding Affinity



### Measuring RMSD on PDBBind results

In [ ]:
def create_rmsd_io_dict(template, prev_dict):
    # Copy the template to avoid modifying the original
    new_dict = template.copy()

    # Replace the filepaths in the new dictionary with the values from the previous dictionary
    new_dict['inputs']['reference_structure']['filepath'] = prev_dict['inputs']['small_molecule']['filepath']
    new_dict['inputs']['comparison_structure']['filepath'] = prev_dict['outputs']['best_docked_small_molecule']['filepath']
    
    return new_dict

def create_rmsd_io_dict_list(template, prev_dict_list):
    new_dict_list = []

    for prev_dict in prev_dict_list:
        # Only process dictionaries where the state is 'completed'
        if prev_dict['state'] == 'completed':
            new_dict = create_rmsd_io_dict(template, prev_dict)
            new_dict_list.append(new_dict)

    return new_dict_list

# Example usage
template = {
    "outputs": {
        "scored_small_molecule": {
            "class": "File",
            "filepath": ""
        },
        "scores": {
            "class": "File",
            "filepath": ""
        }
    },
    "tool": "tools/openbabel/rmsd-openbabel.json",
    "inputs": {
        "reference_structure": {
            "class": "File",
            "filepath": "/Users/rindtorff/github/labdao/plex/testdata/scoring/abl/7n9g.sdf"
        },
        "comparison_structure": {
            "class": "File",
            "filepath": "/Users/rindtorff/github/labdao/plex/testdata/scoring/abl/7n9g_ZINC000003986735_docked.sdf"
        }
    },
    "state": "created",
    "errMsg": ""
}

# prev_dict_list should be the list of dictionaries you provided earlier
new_dict_list = create_rmsd_io_dict_list(template, json_dict)

#for new_dict in new_dict_list:
#    print(new_dict)
print(new_dict_list[:5])

In [ ]:
from plex.sdk import run_local
run_local(new_dict_list)

### Preparing PDBBind Affinity data

In [41]:
# 
!wget https://bafybeicl4suczrx7ql2aayegfz6fjg4fs5kplkl2eaj4n4ieheimscrzoi.ipfs.dweb.link/INDEX_general_PL_data.2020

--2023-05-04 04:16:35--  https://bafybeicl4suczrx7ql2aayegfz6fjg4fs5kplkl2eaj4n4ieheimscrzoi.ipfs.dweb.link/INDEX_general_PL_data.2020
Resolving bafybeicl4suczrx7ql2aayegfz6fjg4fs5kplkl2eaj4n4ieheimscrzoi.ipfs.dweb.link (bafybeicl4suczrx7ql2aayegfz6fjg4fs5kplkl2eaj4n4ieheimscrzoi.ipfs.dweb.link)... 209.94.90.1, 2602:fea2:2::1
Connecting to bafybeicl4suczrx7ql2aayegfz6fjg4fs5kplkl2eaj4n4ieheimscrzoi.ipfs.dweb.link (bafybeicl4suczrx7ql2aayegfz6fjg4fs5kplkl2eaj4n4ieheimscrzoi.ipfs.dweb.link)|209.94.90.1|:443... ^C
     PDB_code Kd/Ki
3zzf     2.20    //
3gww     2.46    //
1w8l     1.80    //
3fqa     2.35    //
1zsb     2.00    //


/tmp/ipykernel_520819/1245211774.py:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(plain_text_file, delim_whitespace=True, skiprows=6, header=None, names=column_names, error_bad_lines=False)
b'Skipping line 2430: expected 8 fields, saw 9\nSkipping line 3421: expected 8 fields, saw 9\nSkipping line 3508: expected 8 fields, saw 9\nSkipping line 8591: expected 8 fields, saw 9\nSkipping line 9201: expected 8 fields, saw 9\nSkipping line 17133: expected 8 fields, saw 9\nSkipping line 17383: expected 8 fields, saw 9\nSkipping line 17434: expected 8 fields, saw 9\nSkipping line 17850: expected 8 fields, saw 9\nSkipping line 18069: expected 8 fields, saw 9\nSkipping line 18293: expected 8 fields, saw 9\nSkipping line 18306: expected 8 fields, saw 9\nSkipping line 18368: expected 8 fields, saw 9\nSkipping line 18749: expected 8 fields, saw 9\nSkipping line 19393: expected 8 fields, 

In [42]:
plain_text_file = 'INDEX_general_PL_data.2020'

# Read the plain text file into a pandas DataFrame
column_names = ["PDB_code", "resolution", "release_year", "-logKd/Ki", "Kd/Ki", "reference", "ligand_name"]
df = pd.read_csv(plain_text_file, delim_whitespace=True, skiprows=6, header=None, names=column_names, error_bad_lines=False)

# Select the first and fifth columns
selected_columns = df[["PDB_code", "Kd/Ki"]]

# Display the selected data
print(df.head())

     PDB_code  resolution  release_year   -logKd/Ki Kd/Ki reference  \
3zzf     2.20        2012          0.40    Ki=400mM    //  3zzf.pdf   
3gww     2.46        2009          0.45  IC50=355mM    //  3gwu.pdf   
1w8l     1.80        2004          0.49    Ki=320mM    //  1w8l.pdf   
3fqa     2.35        2009          0.49  IC50=320mM    //  3fq7.pdf   
1zsb     2.00        1996          0.60    Kd=250mM    //  1zsb.pdf   

     ligand_name  
3zzf       (NLG)  
3gww       (SFX)  
1w8l       (1P3)  
3fqa   (GAB&PMP)  
1zsb       (AZM)  


/tmp/ipykernel_520819/3942543180.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(plain_text_file, delim_whitespace=True, skiprows=6, header=None, names=column_names, error_bad_lines=False)
b'Skipping line 2430: expected 8 fields, saw 9\nSkipping line 3421: expected 8 fields, saw 9\nSkipping line 3508: expected 8 fields, saw 9\nSkipping line 8591: expected 8 fields, saw 9\nSkipping line 9201: expected 8 fields, saw 9\nSkipping line 17133: expected 8 fields, saw 9\nSkipping line 17383: expected 8 fields, saw 9\nSkipping line 17434: expected 8 fields, saw 9\nSkipping line 17850: expected 8 fields, saw 9\nSkipping line 18069: expected 8 fields, saw 9\nSkipping line 18293: expected 8 fields, saw 9\nSkipping line 18306: expected 8 fields, saw 9\nSkipping line 18368: expected 8 fields, saw 9\nSkipping line 18749: expected 8 fields, saw 9\nSkipping line 19393: expected 8 fields, 